In [2]:
import json
import glob
import os
import tqdm
import fsspec
import xarray as xr
import kerchunk.hdf
from kerchunk.combine import MultiZarrToZarr

ModuleNotFoundError: No module named 'kerchunk'

In [ ]:
root_directory = "/work/uc0928/DATA/ocean/aviso_ssh/DATA/"
path_to_output = "/work/ab0995/a270088/"
files = "dt_global_twosat_phy_l4*.nc"

In [ ]:
# Initialize an empty list to store the file paths
file_paths = []

# Iterate over all year and month directories
for year in range(1993, 2023):  # Replace YYYY with the start year and n with the number of years
    for month in range(1, 13):
        # Create a pattern to match all daily files in the current year and month directory
        pattern = os.path.join(root_directory, f"{year}/{month:02d}/*")

        # Use glob to find all files matching the pattern
        files = glob.glob(pattern)
        files.sort()

        # Add the absolute paths to the list
        file_paths.extend([os.path.abspath(file) for file in files])

In [ ]:
urls = file_paths
singles = []
for u in tqdm.tqdm(urls):
    with fsspec.open(u) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=100)
        singles.append(h5chunks.translate())

In [ ]:
mzz = MultiZarrToZarr(
     singles,
    concat_dims=["time"]
)

In [ ]:
with open(f"{path_to_output}/AVISO.json", "w") as outfile:
    json.dump(out, outfile)